1. Configuración de la sesión Spark

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Crear sesión de Spark con configuración para MinIO
spark = SparkSession.builder \
    .appName("OrderLinesBatchProcessing") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()


print("✅ Spark Session creada")
print(f"Versión de Spark: {spark.version}")

Error: LinkageError occurred while loading main class org.apache.spark.launcher.Main
	java.lang.UnsupportedClassVersionError: org/apache/spark/launcher/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0
/home/bigdata/miniconda3/envs/kafka/lib/python3.14/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

2. Lectura de Datos (Filtro por Partición)

In [ ]:
# Para leer TODO el bucket:
path = "s3a://temperature-sensors/"

# Ejemplo para leer solo una hora específica (España):
# path = "s3a://temperature-sensors/year=2026/month=02/day=18/hour=18/"

df = spark.read.json(path)

# Mostrar esquema para verificar que los datos llegan bien
df.printSchema()
df.show(5)

3. Cálculo de Métricas por Sensor

In [ ]:
metrics_df = df.groupBy("sensor_id").agg(
    avg("temperature").alias("temp_promedio"),
    min("temperature").alias("temp_minima"),
    max("temperature").alias("temp_maxima")
)

metrics_df.show()

4. Visualización de la Tendencia

In [ ]:
# Convertimos a Pandas para graficar
# Nota: Si el volumen de datos es masivo, filtra antes de hacer .toPandas()
pdf = df.select("timestamp", "temperature", "sensor_id").orderBy("timestamp").toPandas()

# Convertir string a objeto datetime si es necesario
pdf['timestamp'] = pd.to_datetime(pdf['timestamp'])

plt.figure(figsize=(12, 6))
for sensor in pdf['sensor_id'].unique():
    subset = pdf[pdf['sensor_id'] == sensor]
    plt.plot(subset['timestamp'], subset['temperature'], label=f'Sensor {sensor}')

plt.title("Tendencia de Temperatura a lo Largo del Tiempo")
plt.xlabel("Tiempo (Hora España)")
plt.ylabel("Temperatura ºC")
plt.legend()
plt.grid(True)
plt.show()